In [18]:
import torch
import numpy as np
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim

In [3]:
z= torch.FloatTensor([1,2,3])

In [4]:
hypothesis= F.softmax(z,dim=0)    # exp(i) / sigma exp(i)    class= i
print(hypothesis) 

tensor([0.0900, 0.2447, 0.6652])


In [5]:
z= torch.rand(3,5,requires_grad=True)
hypothesis= F.softmax(z,dim=1)    
print(hypothesis) # this value is predict value

tensor([[0.2160, 0.2008, 0.2087, 0.2175, 0.1570],
        [0.1970, 0.2387, 0.1424, 0.2827, 0.1392],
        [0.1766, 0.2761, 0.1202, 0.1399, 0.2872]], grad_fn=<SoftmaxBackward0>)


In [6]:
y= torch.randint(5,(3,)).long()
print(y)

tensor([2, 2, 0])


In [9]:
y_one_hot= torch.zeros_like(hypothesis)
y_one_hot.scatter_(1,y.unsqueeze(1),1)  # 1 of first factor=dim

tensor([[0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 0.]])

In [11]:
cost= (y_one_hot * (-torch.log(hypothesis))).sum(dim=1).mean() # cross entropy loss
print(cost)    

tensor(1.7499, grad_fn=<MeanBackward0>)


In [12]:
F.log_softmax(z,dim=1) 

tensor([[-1.5324, -1.6054, -1.5669, -1.5257, -1.8514],
        [-1.6248, -1.4324, -1.9490, -1.2633, -1.9720],
        [-1.7338, -1.2871, -2.1190, -1.9665, -1.2475]],
       grad_fn=<LogSoftmaxBackward0>)

In [13]:
F.nll_loss(F.log_softmax(z,dim=1),y) # NLL= negative log likelihood

tensor(1.7499, grad_fn=<NllLossBackward0>)

In [14]:
F.cross_entropy(z,y) # simply

tensor(1.7499, grad_fn=<NllLossBackward0>)

## training with Low-level cross entropy

In [16]:
x_train= [[1,2,1,1],[2,1,3,2],[3,1,3,4],[4,1,5,5],[1,7,5,5],[1,2,5,6],
         [1,6,6,6],[1,7,7,7]]
y_train= [2,2,2,1,1,1,0,0]
x_train= torch.FloatTensor(x_train)
y_train=torch.LongTensor(y_train)

In [20]:
w= torch.zeros((4,3),requires_grad=True)
b= torch.zeros(1, requires_grad=True)
optimizer= optim.SGD([w,b],lr=0.1)
nb_epochs=1000
for epoch in range(nb_epochs + 1):
    
    hypothesis= F.softmax(x_train.matmul(w)+b,dim=1)
    y_one_hot=torch.zeros_like(hypothesis)
    y_one_hot.scatter_(1,y_train.unsqueeze(1),1)
    cost= (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()
    # z=x_train.matmul(w) + b
    # cost= F.cross_entropy(z,y_train) also possible
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 ==0:
        print('epoch {:4d} / {} cost: {:.6f}'.format(epoch,nb_epochs,cost.item()))


epoch    0 / 1000 cost: 1.098612
epoch  100 / 1000 cost: 0.901535
epoch  200 / 1000 cost: 0.839114
epoch  300 / 1000 cost: 0.807826
epoch  400 / 1000 cost: 0.788472
epoch  500 / 1000 cost: 0.774822
epoch  600 / 1000 cost: 0.764449
epoch  700 / 1000 cost: 0.756191
epoch  800 / 1000 cost: 0.749398
epoch  900 / 1000 cost: 0.743671
epoch 1000 / 1000 cost: 0.738749


## using class

In [22]:
class scm(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear= nn.Linear(4,3)
        
    def forward(self,x):
        return self.linear(x)
model= scm() 

In [24]:
optimizer= optim.SGD(model.parameters(),lr=0.1)
nb_epochs=1000
for epoch in range(nb_epochs + 1):
    
    prediction= model(x_train)
    cost= F.cross_entropy(prediction,y_train)
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 ==0:
        print('epoch {:4d} / {} cost: {:.6f}'.format(epoch,nb_epochs,cost.item()))


epoch    0 / 1000 cost: 1.181581
epoch  100 / 1000 cost: 0.641062
epoch  200 / 1000 cost: 0.556476
epoch  300 / 1000 cost: 0.502024
epoch  400 / 1000 cost: 0.457680
epoch  500 / 1000 cost: 0.418170
epoch  600 / 1000 cost: 0.381115
epoch  700 / 1000 cost: 0.344991
epoch  800 / 1000 cost: 0.308668
epoch  900 / 1000 cost: 0.271994
epoch 1000 / 1000 cost: 0.243457
